Pandas Practice

In [10]:
import pandas as pd

data = {
    "cal": [420, 380, 290],
    "dur": [50,40,45]
}

df = pd.DataFrame(data)

print(df)                   #Print dataframe
print("")
print(df.loc[0])            #Print series
print("")
print(df.loc[[0]])          #Print same but as dataframe
print("")
print(df.loc[[0,1]])        #Print Slice of data frame, inclusive, slice can not be printed as series

   cal  dur
0  420   50
1  380   40
2  290   45

cal    420
dur     50
Name: 0, dtype: int64

   cal  dur
0  420   50

   cal  dur
0  420   50
1  380   40


In [19]:
data = {
    "cal": [420, 380, 290],
    "dur": [50,40,45],
    "date": ["12121212", None, None]
}

df = pd.DataFrame(data)
print(df)
print("")

df.fillna( {"date": "11/11/11"}, inplace=True)
df["date"] = pd.to_datetime(df["date"], format='mixed')
print(df)

   cal  dur      date
0  420   50  12121212
1  380   40       NaN
2  290   45       NaN

   cal  dur       date
0  420   50 1212-12-12
1  380   40 2011-11-11
2  290   45 2011-11-11


In [22]:
data = {
    "cal": [420, 380, 290],
    "dur": [50,40,45]
}
df = pd.DataFrame(data)
print(df.corr())

          cal       dur
cal  1.000000  0.300376
dur  0.300376  1.000000


Pandas Data Cleaning Challenge

In [ ]:
import pandas as pd
import numpy as np

#Drop rows with invalid data in column
def drop_rows(df:pd.DataFrame, columns:list[str], invalid_data:list):
    for column in columns:
        for x in df.index:
            if(df.loc[x, column] in invalid_data):
                df.drop(x, inplace=True)

#Fix rows with invalid data in column
def fix_rows(df:pd.DataFrame, columns:list[str], invalid_to_correct_data:dict):
    for column in columns:
        for x in df.index:
            if( df.loc[x, column] in invalid_to_correct_data):
                df.loc[x, column] = invalid_to_correct_data[df.loc[x, column]]

#Move data in column to within acceptable range (inclusive)
def clamp_int_data(df:pd.DataFrame, columns:list[str], minimum:int, maximum:int):
    for column in columns:
        for x in df.index:
            if (type(df.loc[x, column]) is int and df.loc[x, column] < minimum):
                df.loc[x, column] = minimum
            elif(type(df.loc[x, column]) is int and df.loc[x, column] > maximum):
                df.loc[x, column] = maximum

#If data in column is a string, convert it to title
def format_as_title(df:pd.DataFrame, columns:list[str]):
    for column in columns:
        for x in df.index:
            if(type(df.loc[x, column]) is str):
                df.loc[x, column] = df.loc[x, column].title()

df = pd.read_json("books_dirty.json")

#1
empty_to_NaN = {"": np.nan}
columns = ["last_checkout"]
fix_rows(df, columns, empty_to_NaN)

#2
minimum = 1800
maximum = 2026
columns = ["publication_year"]
clamp_int_data(df, columns, minimum, maximum)

#3
#Replacing all incorrect values with NaN
columns = ["publication_year", "page_count", "average_rating", "ratings_count", "price_usd", "in_print", "sales_millions", "last_checkout", "available"]
incorrect_to_NaN = {"Unknown": np.nan, "N/A": np.nan, "Unknown": np.nan}
fix_rows(df, columns, incorrect_to_NaN)

#Fixing "true"/"false" to actual boolean values
columns = ["in_print", "available"]
string_to_bool = {"true": True, "false": False}
fix_rows(df, columns, string_to_bool)

#Clamp negative values
minimum = 0
maximum = 999999999
columns = ["page_count", "price_usd", "sales_millions"]
clamp_int_data(df, columns, minimum, maximum)

#4
df["last_checkout"] = pd.to_datetime(df["last_checkout"], format="mixed")

#5
columns = ["genre", "language", "format", "publisher"]
format_as_title(df, columns)

#6
df.drop_duplicates(subset=["author", "title"], inplace=True)

#7
#All or almost all values have a null somewhere so this last step deletes everything
df.dropna(inplace=True)

print(df.info())



<class 'pandas.DataFrame'>
Index: 330 entries, 0 to 499
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   book_id           330 non-null    str           
 1   title             330 non-null    str           
 2   author            330 non-null    str           
 3   genre             330 non-null    str           
 4   publication_year  124 non-null    object        
 5   page_count        171 non-null    object        
 6   average_rating    120 non-null    object        
 7   ratings_count     120 non-null    object        
 8   price_usd         105 non-null    object        
 9   publisher         330 non-null    str           
 10  language          330 non-null    str           
 11  format            330 non-null    str           
 12  in_print          260 non-null    object        
 13  sales_millions    93 non-null     object        
 14  last_checkout     90 non-null     datetime